In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import json
import re
from selenium import webdriver

In [166]:
def get_page_details(response):
    soup = BeautifulSoup(response.text)
    all_script = soup.find_all('script')
    script_filter = [i for i in all_script if len(i.attrs) == 0]
    con = script_filter[2].contents[0][19:]
    new_con = json.loads(con)
    return new_con['properties']

In [167]:
def get_web_details(page):
    if page == 0:
        response = requests.get('https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
        return get_page_details(response)
    else:
        response = requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&index={page}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
        return get_page_details(response)



In [55]:
postCode_regex = '([A-Z]+\d+[A-Z]*)'
listing_sources = ['Rightmove', 'Zoopla', 'On The Market(OTM)']
parent_url = 'https://www.rightmove.co.uk'

pages_dict = {'transaction_type':[], 'bedrooms':[], 'bathrooms':[], 'description':[], 'property_type':[], 'price':[], 'location':[],
           'post_code':[], 'latitude':[],'longitude':[], 'agent':[], 'agent_contact':[], 'listing_source':[], 'listing_url':[], 'date':[]}

In [175]:

pages_df = pd.DataFrame()

for i in range(0, 985, 24):
    page_details = get_web_details(i)
    for j in page_details:
        pages_dict['transaction_type'].append(j['transactionType'])
        pages_dict['bedrooms'].append(j['bedrooms'])
        pages_dict['bathrooms'].append(j['bathrooms'])
        pages_dict['description'].append(j['summary'])
        pages_dict['property_type'].append(j['propertySubType'])
        pages_dict['price'].append(j['price']['amount'])
        pages_dict['location'].append(j['displayAddress'])
        pages_dict['post_code'].append(None if re.search(postCode_regex, j['displayAddress']) is None else re.search(postCode_regex, j['displayAddress'])[0])
        pages_dict['latitude'].append(j['location']['latitude'])
        pages_dict['longitude'].append(j['location']['longitude'])
        pages_dict['agent'].append(j['customer']['brandTradingName'])
        pages_dict['agent_contact'].append(j['customer']['contactTelephone'])
        pages_dict['listing_source'].append(listing_sources[0])
        pages_dict['listing_url'].append(parent_url + j['propertyUrl'])
        pages_dict['date'].append(j['addedOrReduced'].split(" ").pop())
    
pages_df.append(pages_dict, ignore_index=True)
pages_df.head(20)
#df = pd.DataFrame()



C:\Users\AKINDE KOLAWOLE\AppData\Local\Temp\ipykernel_18040\3009056639.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pages_df.append(pages_dict, ignore_index=True)


""


In [65]:
pages_df = pages_df.append(pd.DataFrame(pages_dict), ignore_index=True)
pages_df.head(5)

NameError: name 'pages_df' is not defined

In [198]:
sales_response = requests.get('https://www.rightmove.co.uk/property-to-rent/find.html?searchType=RENT&locationIdentifier=REGION%5E87490&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&sortByPriceDescending=&_includeLetAgreed=on&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&letType=&letFurnishType=&houseFlatShare=')

200

In [190]:
sales_soup = get_page_details(sales_response)
sales_soup

[{'id': 85929765,
  'bedrooms': 2,
  'bathrooms': 2,
  'numberOfImages': 11,
  'numberOfFloorplans': 1,
  'numberOfVirtualTours': 0,
  'summary': 'An exceptionally spacious 2 bedroom luxurious apartment set within a new sought after development.',
  'displayAddress': 'Kensington Gardens Square London W2',
  'countryCode': 'GB',
  'location': {'latitude': 51.51512, 'longitude': -0.19002},
  'propertyImages': {'images': [{'srcUrl': 'https://media.rightmove.co.uk:443/dir/crop/10:9-16:9/51k/50074/85929765/50074_P254646_IMG_14_0000_max_476x317.jpeg',
     'url': '51k/50074/85929765/50074_P254646_IMG_14_0000.jpeg',
     'caption': 'View'},
    {'srcUrl': 'https://media.rightmove.co.uk:443/dir/crop/10:9-16:9/51k/50074/85929765/50074_P254646_IMG_00_0000_max_476x317.jpeg',
     'url': '51k/50074/85929765/50074_P254646_IMG_00_0000.jpeg',
     'caption': 'Living Room '},
    {'srcUrl': 'https://media.rightmove.co.uk:443/dir/crop/10:9-16:9/51k/50074/85929765/50074_P254646_IMG_13_0001_max_476x317.j

In [193]:
sd = '£3,467 pcm'
regex = '\d+,\d+'
find = re.search(regex, sd)[0].replace(",", "")
find

'3467'

In [54]:
driver = webdriver.Firefox()
driver.get('https://www.zoopla.co.uk/for-sale/property/london/?q=London&search_source=home')

html = driver.page_source
driver.close()
soups = BeautifulSoup(html)

In [98]:
def getBedAndBathroomData(value, item):
    for i in value:
        if list(i.values()).__contains__(item):
            return int(i['content'])
    return None

In [115]:
def getPrice(value):
    price_regex = '\d+,?\d*,?\d*,?\d*'
    return int(re.search(price_regex, value)[0].replace(",",""))


In [124]:
parent_url_zoopla = 'https://www.zoopla.co.uk/'
listing_source_zoopla = 'Zoopla'

pages_dict_zoopla = {'transaction_type':[], 'bedrooms':[], 'bathrooms':[], 'description':[], 'property_type':[], 'price':[], 'location':[],
           'post_code':[], 'latitude':[],'longitude':[], 'agent':[], 'agent_contact':[], 'listing_source':[], 'listing_url':[], 'date':[]}

In [104]:
'''zoopla_content = soups.find("script", attrs={'id':'__NEXT_DATA__'}).contents[0]
zoopla_dict = json.loads(zoopla_content)
property_list = zoopla_dict['props']['pageProps']['regularListingsFormatted']

for j in property_list:
    pages_dict_zoopla['transaction_type'].append(j['listingUris']['detail'][5:9])
    pages_dict_zoopla['bedrooms'].append(getBedAndBathroomData(j['features'], 'bed'))
    pages_dict_zoopla['bathrooms'].append(getBedAndBathroomData(j['features'], 'bath'))
    pages_dict_zoopla['description'].append(j['summaryDescription'])
    pages_dict_zoopla['property_type'].append(j['propertyType'])
    pages_dict_zoopla['price'].append(getPrice(j['price']))
    pages_dict_zoopla['location'].append(j['address'])
    pages_dict_zoopla['post_code'].append(None if re.search(postCode_regex, j['address']) is None else re.search(postCode_regex, j['address'])[0])
    pages_dict_zoopla['latitude'].append(j['location']['coordinates']['latitude'])
    pages_dict_zoopla['longitude'].append(j['location']['coordinates']['longitude'])
    pages_dict_zoopla['agent'].append(j['branch']['name'])
    pages_dict_zoopla['agent_contact'].append(j['branch']['phone'])
    pages_dict_zoopla['listing_source'].append(listing_source_zoopla)
    pages_dict_zoopla['listing_url'].append(parent_url_zoopla + j['listingUris']['detail'])
    pages_dict_zoopla['date'].append(j['lastPublishedDate'][0:10])

#new_dict = {a:len(pages_dict_zoopla[a]) for a in pages_dict_zoopla}
new_df = pd.DataFrame(pages_dict_zoopla)'''


In [112]:
driver_rent = webdriver.Firefox()
driver_rent.get('https://www.zoopla.co.uk/to-rent/property/london/?price_frequency=per_month&q=London&search_source=home')

html_rent = driver_rent.page_source
driver_rent.close()
soup_rent = BeautifulSoup(html_rent)

In [123]:
content = soup_rent.find("script", attrs={'id':'__NEXT_DATA__'}).contents[0]
zoopla_dict = json.loads(content)
property_list = zoopla_dict['props']['pageProps']['initialState']['seoAccordions'][0]['transactionType']
property_list

'rent'

In [125]:
zoopla_content = soup_rent.find("script", attrs={'id':'__NEXT_DATA__'}).contents[0]
zoopla_dict = json.loads(zoopla_content)
property_list = zoopla_dict['props']['pageProps']['regularListingsFormatted']
transaction_type = zoopla_dict['props']['pageProps']['initialState']['seoAccordions'][0]['transactionType']

for j in property_list:
    pages_dict_zoopla['transaction_type'].append(transaction_type)
    pages_dict_zoopla['bedrooms'].append(getBedAndBathroomData(j['features'], 'bed'))
    pages_dict_zoopla['bathrooms'].append(getBedAndBathroomData(j['features'], 'bath'))
    pages_dict_zoopla['description'].append(j['summaryDescription'])
    pages_dict_zoopla['property_type'].append(j['propertyType'])
    pages_dict_zoopla['price'].append(getPrice(j['price']))
    pages_dict_zoopla['location'].append(j['address'])
    pages_dict_zoopla['post_code'].append(None if re.search(postCode_regex, j['address']) is None else re.search(postCode_regex, j['address'])[0])
    pages_dict_zoopla['latitude'].append(j['location']['coordinates']['latitude'])
    pages_dict_zoopla['longitude'].append(j['location']['coordinates']['longitude'])
    pages_dict_zoopla['agent'].append(j['branch']['name'])
    pages_dict_zoopla['agent_contact'].append(j['branch']['phone'])
    pages_dict_zoopla['listing_source'].append(listing_source_zoopla)
    pages_dict_zoopla['listing_url'].append(parent_url_zoopla + j['listingUris']['detail'])
    pages_dict_zoopla['date'].append(j['lastPublishedDate'][0:10])

#new_dict = {a:len(pages_dict_zoopla[a]) for a in pages_dict_zoopla}
new_df = pd.DataFrame(pages_dict_zoopla)

In [127]:
new_df.head(5)

,transaction_type,bedrooms,bathrooms,description,property_type,price,location,post_code,latitude,longitude,agent,agent_contact,listing_source,listing_url,date
0,rent,5.0,2.0,"Presenting a refurbished, luxurious flat to re...",flat,8082,"Park Road, London NW8",NW8,51.529670,-0.167630,Scraye,020 3641 8040,Zoopla,https://www.zoopla.co.uk//to-rent/details/6451...,2023-04-29
1,rent,5.0,3.0,"Presenting a stunning flat to rent in St, John...",flat,8450,"Park Road, London NW8",NW8,51.529670,-0.167630,Scraye,020 3641 8040,Zoopla,https://www.zoopla.co.uk//to-rent/details/6451...,2023-04-29
2,rent,3.0,2.0,"3 Bed Flat, Gipsy Hill, SE19 We are proud to o...",flat,2100,"Gipsy Hill, London SE19",SE19,51.421374,-0.083734,OpenRent,020 3542 2178,Zoopla,https://www.zoopla.co.uk//to-rent/details/6451...,2023-04-29
3,rent,3.0,NaN,This stunning ground floor apartment has been ...,flat,4983,"King Street, London W6",W6,51.493675,-0.239744,Residential Land,020 3463 6976,Zoopla,https://www.zoopla.co.uk//to-rent/details/3838...,2023-04-29
4,rent,2.0,NaN,This stunning newly restored and extended two ...,flat,3315,"King Street, London W6",W6,51.493675,-0.239744,Residential Land,020 3463 6976,Zoopla,https://www.zoopla.co.uk//to-rent/details/5810...,2023-04-29


In [2]:
#ON THE MARKET DATA SCRAPING
market = webdriver.Firefox()
market.get('https://www.onthemarket.com/for-sale/property/london/?view=grid')

html_market = market.page_source
market.close()
soup_market = BeautifulSoup(html_market)

In [6]:
pages_dict_otm = {'transaction_type':[], 'bedrooms':[], 'bathrooms':[], 'description':[], 'property_type':[], 'price':[], 'location':[],
           'post_code':[], 'latitude':[],'longitude':[], 'agent':[], 'agent_contact':[], 'listing_source':[], 'listing_url':[], 'date':[]}

In [3]:
def find_required_tag(element):
    return element.has_attr('type') and repr(element.string).__contains__('__OTM__.jsonData')

In [4]:
soup_market.contents
extracted_string = soup_market.find(find_required_tag).string

In [5]:
extract = extracted_string[extracted_string.index('__OTM__.jsonData') + 19 : extracted_string.index('__OTM__.globals') - 1].strip(';\n')
otm_dict = json.loads(extract)['properties']
otm_dict

[{'features': ['Tenure: Leasehold',
   'Large penthouse-Approx. 3500 Sq ft',
   'Roof terrace with river views of Boat Race finishing line',
   'Currently arranged as two flats'],
  'bedrooms-text': '4',
  'images-count': 11,
  'price-qualifier': None,
  'images': [{'default': 'https://media.onthemarket.com/properties/8762174/1442907468/image-0-480x320.jpg',
    'portrait?': False,
    'webp': 'https://media.onthemarket.com/properties/8762174/1442907468/image-0-480x320.webp'},
   {'default': 'https://media.onthemarket.com/properties/8762174/1430370549/image-1-480x320.jpg',
    'portrait?': False,
    'webp': 'https://media.onthemarket.com/properties/8762174/1430370549/image-1-480x320.webp'},
   {'default': 'https://media.onthemarket.com/properties/8762174/1442907468/image-2-480x320.jpg',
    'portrait?': False,
    'webp': 'https://media.onthemarket.com/properties/8762174/1442907468/image-2-480x320.webp'},
   {'default': 'https://media.onthemarket.com/properties/8762174/1428724141/imag

In [ ]:
for j in otm_dict:
    pages_dict_otm['transaction_type'].append(transaction_type)
    pages_dict_otm['bedrooms'].append(getBedAndBathroomData(j['features'], 'bed'))
    pages_dict_otm['bathrooms'].append(getBedAndBathroomData(j['features'], 'bath'))
    pages_dict_zoopla['description'].append(j['summaryDescription'])
    pages_dict_zoopla['property_type'].append(j['propertyType'])
    pages_dict_zoopla['price'].append(getPrice(j['price']))
    pages_dict_zoopla['location'].append(j['address'])
    pages_dict_zoopla['post_code'].append(None if re.search(postCode_regex, j['address']) is None else re.search(postCode_regex, j['address'])[0])
    pages_dict_zoopla['latitude'].append(j['location']['coordinates']['latitude'])
    pages_dict_zoopla['longitude'].append(j['location']['coordinates']['longitude'])
    pages_dict_zoopla['agent'].append(j['branch']['name'])
    pages_dict_zoopla['agent_contact'].append(j['branch']['phone'])
    pages_dict_zoopla['listing_source'].append(listing_source_zoopla)
    pages_dict_zoopla['listing_url'].append(parent_url_zoopla + j['listingUris']['detail'])
    pages_dict_zoopla['date'].append(j['lastPublishedDate'][0:10])

In [4]:
market_personal = webdriver.Firefox()
market_personal.get('https://www.onthemarket.com/details/13151588/')

html_personal = market_personal.page_source
market_personal.close()
soup_personal = BeautifulSoup(html_personal)

In [5]:
required_tag = soup_personal.find(find_required_tag).string
required_tag

'\n window.__OTM__ = {\n   currentQuery: {},\n }\n\n __OTM__.jsonData = {"station":[{"name":"St Margarets (London)","full-name":"St Margarets (London) Station","all-lines":[{"full-name":""}],"all-networks":[{"type":"default-network"}],"location_type":"station","id":"station-2122","display-distance":"0.4mi."},{"name":"Richmond (London)","full-name":"Richmond (London) Station","all-lines":[{"full-name":""}],"all-networks":[{"type":"default-network"}],"location_type":"station","id":"station-1895","display-distance":"0.6mi."},{"name":"Richmond Underground","full-name":"Richmond Underground Station","all-lines":[{"full-name":"District ","short-name":"district"},{"full-name":" Overground","short-name":"overground"}],"all-networks":[{"type":"Tube"}],"location_type":"station","id":"station-3143","display-distance":"0.6mi."}],"rooms":{"descriptions":[]},"description":"Arranged over two floors and measuring 2,712 sq ft approximately, this unique property which was newly constructed in 2012, prov

In [7]:
new_extract = required_tag[required_tag.index('__OTM__.jsonData') + 19 : required_tag.index('__OTM__.globals') - 1].strip(';\n')
json.loads(new_extract)


{'station': [{'name': 'St Margarets (London)',
   'full-name': 'St Margarets (London) Station',
   'all-lines': [{'full-name': ''}],
   'all-networks': [{'type': 'default-network'}],
   'location_type': 'station',
   'id': 'station-2122',
   'display-distance': '0.4mi.'},
  {'name': 'Richmond (London)',
   'full-name': 'Richmond (London) Station',
   'all-lines': [{'full-name': ''}],
   'all-networks': [{'type': 'default-network'}],
   'location_type': 'station',
   'id': 'station-1895',
   'display-distance': '0.6mi.'},
  {'name': 'Richmond Underground',
   'full-name': 'Richmond Underground Station',
   'all-lines': [{'full-name': 'District ', 'short-name': 'district'},
    {'full-name': ' Overground', 'short-name': 'overground'}],
   'all-networks': [{'type': 'Tube'}],
   'location_type': 'station',
   'id': 'station-3143',
   'display-distance': '0.6mi.'}],
 'rooms': {'descriptions': []},
 'description': 'Arranged over two floors and measuring 2,712 sq ft approximately, this unique